In [1]:
%load_ext autoreload

In [10]:
import sys
import os
import json
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split

os.environ['CUDA_VISIBLE_DEVICES']='2'
pd.set_option('display.max_colwidth', None)
sys.path.append('/workspace/computationally-undermining-arguments/scripts/')
sys.path.append('/workspace/computationally-undermining-arguments/src-py/')
sys.path.append('/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/')

In [46]:
%autoreload
import interact
import train
import utility

In [28]:
#_, tokenizer = interact.load_model('/workspace/ceph_data/argument-undermining/models/gen_models/Nov02_10-06-11_gammaweb02_full_post_full_counter/')

### Data Preparation:

In [14]:
training_df =  pd.read_json("/workspace/ceph_data/argument-undermining/data/gen_training.json")
valid_df =  pd.read_json("/workspace/ceph_data/argument-undermining/data/gen_validation.json")

In [5]:
# #for long counter-premises
# training_df =  pd.read_json("/workspace/ceph_data/argument-undermining/data/new_gen_training.json")
# valid_df =  pd.read_json("/workspace/ceph_data/argument-undermining/data/new_gen_validation.json")

In [6]:
training_df = clean_df(training_df)
valid_df = clean_df(valid_df)

In [14]:
training_data1 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='title', wrong_counter='full_post')
json.dump(training_data1, open('/workspace/ceph_data/argument-undermining/data/gpt_data/title_premise.json', 'w'))

training_data2 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='full_post', wrong_counter='full_post')
json.dump(training_data2, open('/workspace/ceph_data/argument-undermining/data/gpt_data/full_post_premise.json', 'w'))

training_data3 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='title', wrong_counter='full_post')
json.dump(training_data3, open('/workspace/ceph_data/argument-undermining/data/gpt_data/title_full_counter.json', 'w'))

training_data4 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='full_post', wrong_counter='full_post')
json.dump(training_data4, open('/workspace/ceph_data/argument-undermining/data/gpt_data/full_post_full_counter.json', 'w'))

training_data5 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='title+full_post', wrong_counter='full_post')
json.dump(training_data5, open('/workspace/ceph_data/argument-undermining/data/gpt_data/argument_premise.json', 'w'))

training_data6 = prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='title+full_post', wrong_counter='full_post')
json.dump(training_data6, open('/workspace/ceph_data/argument-undermining/data/gpt_data/argument_full_counter.json', 'w'))

#### Preparing Jo Data:

In [43]:
jo_df =  pd.read_pickle("/workspace/ceph_data/argument-undermining/jo_data/gen_data.pickle")

training_df = jo_df[jo_df.split =='train']
valid_df = jo_df[jo_df.split =='val']

In [44]:
training_df = utility.clean_df(training_df)
valid_df    = utility.clean_df(valid_df)

/workspace/computationally-undermining-arguments/src-py/utility.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[title_clm] = df[title_clm].apply(lambda x : x.strip().lower())
/workspace/computationally-undermining-arguments/src-py/utility.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[post_clm] = df[post_clm].apply(lambda x : [sent.strip().lower() for sent in x])


In [47]:
jo_training = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df[0:5000]},
                                                full_counter=False, context='title+post', 
                                                post_clm='post', 
                                                comment_clm='comment_sents', 
                                                attacks_clm='premise_counter_premise_pairs', max_sens=20, baseline=False)

In [51]:
json.dump(jo_training, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', 'w'))

In [10]:
# training_data1 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='title')
# json.dump(training_data1, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_title_premise.json', 'w'))

# training_data2 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='full_post')
# json.dump(training_data2, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_full_post_premise.json', 'w'))

# training_data3 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='title')
# json.dump(training_data3, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_title_full_counter.json', 'w'))

# training_data4 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='full_post')
# json.dump(training_data4, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_full_post_full_counter.json', 'w'))

# training_data5 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=False, context='title+full_post')
# json.dump(training_data5, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_argument_premise.json', 'w'))

# training_data6 = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df},  full_counter=True, context='title+full_post')
# json.dump(training_data6, open('/workspace/ceph_data/argument-undermining/data/gpt_data/jo_argument_full_counter.json', 'w'))

### Training GPT2:

In [76]:
! CUDA_VISIBLE_DEVICES=3 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/argument_full_counter.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --output_model_checkpoint argument_full_counter \
                    --dataset_cache ./argument_full_counter_cache \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 3

  File "/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py", line 85
    if premise_as_str.find(s_str) > -1 or s_str.find(premise_as_str) > -1:
                                                                         ^
SyntaxError: invalid syntax


In [ ]:
! CUDA_VISIBLE_DEVICES=4 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/argument_premise.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --dataset_cache ./argument_premise_counter_cache \
                    --output_model_checkpoint argument_premise_counter \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 3

In [ ]:
! CUDA_VISIBLE_DEVICES=1 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/full_post_premise.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --dataset_cache ./full_post_premise_counter_cache \
                    --output_model_checkpoint full_post_premise_counter \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 3

In [ ]:
! CUDA_VISIBLE_DEVICES=1 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/title_premise.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --dataset_cache ./title_premise_counter_cache \
                    --output_model_checkpoint title_premise_counter \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 3

In [ ]:
! CUDA_VISIBLE_DEVICES=4 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/full_post_full_counter.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --dataset_cache ./full_post_full_counter_cache \
                    --output_model_checkpoint full_post_full_counter \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 3

### Training model where weak premise tokens are annotated inside the post:

In [ ]:
! CUDA_VISIBLE_DEVICES=5 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --output_model_checkpoint my_approach_v2 \
                    --dataset_cache ./jo_data_cache \
                    --build_instance_version 'v2' \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 6

2020-12-10 21:49:37.025234: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Arguments: Namespace(baseline=False, build_instance_version='v2', dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='openai-gpt', n_epochs=6, num_candidates=2, output_model_checkpoint='my_approach_v2', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Prepare tokenizer, pretrained model and opt

In [ ]:
! CUDA_VISIBLE_DEVICES=3 python /workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json \
                    --model_checkpoint openai-gpt \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --output_model_checkpoint my_approach_v4 \
                    --dataset_cache ./jo_data_cache \
                    --build_instance_version 'v4' \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 6

2020-12-21 17:09:26.828200: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Arguments: Namespace(baseline=False, build_instance_version='v4', dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='openai-gpt', n_epochs=6, num_candidates=2, output_model_checkpoint='my_approach_v4', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Prepare tokenizer, pretrained model and opt